# eBay German Car Classifieds Analysis
In this analysis project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle by user orgesleka.
The original dataset isn't available on Kaggle anymore, but you can find it [here](https://data.world/data-society/used-cars-data).

The data dictionary provided with data is as follows:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings.

In [2]:
# import required modules
import pandas as pd
import numpy as np
import datetime as dt

In [5]:
# read in the data
autos = pd.read_csv('data/autos.csv', encoding='Latin-1')

In [6]:
# quick look at the types in the data, and its columns
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

In [7]:
autos.shape

(371528, 20)

In [8]:
autos.head(4)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17


From the cells above, we can make the following observations:

- The dataset contains 371528 rows and 20 columns, most of which are strings.
- Some columns have null values, but none have more than ~20% null values.
- The column names use camelcase instead of Python's preferred snakecase.

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [9]:
autos.rename(columns={'dateCrawled':'date_crawled','offerType':'offer_type','vehicleType':'vehicle_type','fuelType':'fuel_type','nrOfPictures':'pictures','postalCode':'postal_code','lastSeen':'last_seen','yearOfRegistration':'registration_year', 'monthOfRegistration':'registration_month','notRepairedDamage':'unrepared_damage','dateCreated':'ad_created'}, inplace=True)

Now, to do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for:

- Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.
- Examples of numeric data stored as text which can be cleaned and converted.

In [20]:
# drop the 'abtest' and 'pictures' columns, not necessary for this analysis
to_drop = ['abtest','pictures']
autos = autos.drop(to_drop, axis=1)

# change the 'date_crawled', 'ad_created', and 'last_seen' columns to datetime
time_parse = ['date_crawled','ad_created', 'last_seen']
for col in time_parse:
    autos[col] = pd.to_datetime(autos[col])

# change the 'price', 'kilometer', and 'powerPS' column types to integers
autos['price'] = autos['price'].astype(int)
autos['kilometer'] = autos['kilometer'].astype(int)
autos['powerPS'] = autos['powerPS'].astype(int)

There are a number of text columns where almost all of the values are the same (seller and offer_type). We also converted the price and odometer columns to numeric types.

Let's continue exploring the data, specifically looking for data that doesn't look right. We'll start by analyzing the kilometer and price columns. Here's the steps we'll take:

- Analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.
- Translate German words into English

In [33]:
# check the min and max for price and kilometer
min_price = autos['price'].min()
max_price = autos['price'].max()
min_odo = autos['kilometer'].min()
max_odo = autos['kilometer'].max()
    
print('Min price: {}'.format(min_price))
print('Max price: {}'.format(max_price))
print('Min odo: {}'.format(min_odo))
print('Max odo: {}'.format(max_odo))
print('Price shape: {}'.format(autos['price'].unique().shape))
print('Odo shape: {}'.format(autos['kilometer'].unique().shape))

# translate German into English
translate_gearbox = {'manuell':'manual','automatik':'auto', 'nan':'nan'}
autos['gearbox'] = autos['gearbox'].map(translate_gearbox)

translate_seller = {'privat':'private','gewerblich':'commercial'}
autos['seller'] = autos['seller'].map(translate_seller)

translate_offer = {'Angebot':'Offer','Gesuch':'Request'}
autos['offer_type'] = autos['offer_type'].map(translate_offer)

translate_type = {'nan':'nan', 'coupe':'coupe', 'suv':'suv', 'kleinwagen':'small car', 'limousine':'limousine', 'cabrio':'cabriolet', 'bus':'bus',
       'kombi':'combi', 'andere':'others'}
autos['vehicle_type'] = autos['vehicle_type'].map(translate_type)

translate_damage = {'nan':'nan', 'ja':'yes', 'nein':'no'}
autos['unrepared_damage'] = autos['unrepared_damage'].map(translate_damage)

translate_fuel = {'benzin':'gasoline', 'diesel':'diesel', 'nan':'nan', 'lpg':'lpg', 'andere':'other', 'hybrid':'hybrid', 'cng':'cng',
       'elektro':'electric'}
autos['fuel_type'] = autos['fuel_type'].map(translate_fuel)

Min price: 0
Max price: 2147483647
Min odo: 5000
Max odo: 150000
Price shape: (5597,)
Odo shape: (13,)


In [74]:
# preview the data with prices set to 0 and 2147483647
autos.loc[autos['price'] == 0].append(autos.loc[autos['price'] == 2147483647])

,date_crawled,name,seller,offer_type,price,vehicle_type,registration_year,gearbox,powerPS,model,kilometer,registration_month,fuel_type,brand,unrepared_damage,ad_created,postal_code,last_seen
7,2016-03-21 18:54:38,VW_Derby_Bj_80__Scheunenfund,private,Offer,0,limousine,1980,manual,50,andere,40000,7,gasoline,volkswagen,no,2016-03-21,19348,2016-03-25 16:47:58
40,2016-03-26 22:06:17,Suche_Opel_corsa_a_zu_verschenken,private,Offer,0,NaN,1990,NaN,0,corsa,150000,1,gasoline,opel,NaN,2016-03-26,56412,2016-03-27 17:43:34
115,2016-03-19 18:40:12,Golf_IV_1.4_16V,private,Offer,0,NaN,2017,manual,0,golf,5000,12,gasoline,volkswagen,NaN,2016-03-19,21698,2016-04-01 08:47:05
119,2016-03-20 18:53:27,Polo_6n_Karosse_zu_verschenken,private,Offer,0,small car,1999,NaN,0,NaN,5000,0,gasoline,volkswagen,NaN,2016-03-20,37520,2016-04-07 02:45:22
157,2016-03-11 18:55:53,Opel_meriva_1.6_16_v_lpg__z16xe_no_OPC,private,Offer,0,bus,2004,manual,101,meriva,150000,10,lpg,opel,yes,2016-03-11,27432,2016-03-12 23:47:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371392,2016-03-20 14:55:07,Ford_Fiesta_1.3___60PS___Bj_2002___Klima___Servo,private,Offer,0,small car,2002,manual,60,fiesta,150000,3,gasoline,ford,NaN,2016-03-20,33659,2016-04-06 18:45:23
371402,2016-03-24 13:48:05,Suzuki_Swift_zu_verkaufen,private,Offer,0,small car,1999,manual,53,swift,150000,3,gasoline,suzuki,NaN,2016-03-24,42329,2016-04-07 05:17:24
371431,2016-03-10 22:55:50,Seat_Arosa,private,Offer,0,small car,1999,manual,37,arosa,150000,7,gasoline,seat,yes,2016-03-10,22559,2016-03-12 23:46:32
371522,2016-03-21 09:50:58,Mitsubishi_Cold,private,Offer,0,NaN,2005,manual,0,colt,150000,7,gasoline,mitsubishi,yes,2016-03-21,2694,2016-03-21 10:42:49


Let's now move on to the date columns and understand the date range the data covers.

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. The other two columns are represented as numeric values, so we can use methods like Series.describe() to understand the distribution without any extra data processing.

In [42]:
# preview the 'date_crawled', 'ad_created', and 'last_seen' columns
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-24 11:52:17,2016-03-24,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31,2016-04-06 10:17:21


In [43]:
# get the months the data was collected
pd.DatetimeIndex(autos['date_crawled']).month.value_counts(normalize=True, dropna=False)

3    0.836852
4    0.163148
Name: date_crawled, dtype: float64

We see that the data was collected mostly in March of 2016

One thing that stands out from the exploration we did in the last screen is that the registration_year column contains some odd values:

- The minimum value is 1000, before cars were invented
- The maximum value is 9999, many years into the future

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2020 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [44]:
autos['registration_year'].describe()

count    371528.000000
mean       2004.577997
std          92.866598
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

In [49]:
below_1900 = autos[autos['registration_year'] < 1900]
above_2016 = autos[autos['registration_year'] > 2016]
print(f'Registered Before 1900 : {below_1900.shape[0]}')
print(f'Registered After 2016  : {above_2016.shape[0]}')

Registered Before 1900 : 68
Registered After 2016  : 14680


We will purge entries in the dataset for registered vehicles before 1900 and after 2016. The reason being, it is unlikely that a vehicle was registered that far back in the past and in the future.

In [51]:
# slice for vehicles registered between 1900 and 2016
auto_valid = autos[autos['registration_year'].between(1900,2016)]
print(auto_valid.shape)
auto_valid.head()

(356780, 18)


,date_crawled,name,seller,offer_type,price,vehicle_type,registration_year,gearbox,powerPS,model,kilometer,registration_month,fuel_type,brand,unrepared_damage,ad_created,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,private,Offer,480,NaN,1993,manual,0,golf,150000,0,gasoline,volkswagen,NaN,2016-03-24,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,private,Offer,18300,coupe,2011,manual,190,NaN,125000,5,diesel,audi,yes,2016-03-24,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",private,Offer,9800,suv,2004,auto,163,grand,125000,8,diesel,jeep,NaN,2016-03-14,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,private,Offer,1500,small car,2001,manual,75,golf,150000,6,gasoline,volkswagen,no,2016-03-17,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,private,Offer,3600,small car,2008,manual,69,fabia,90000,7,diesel,skoda,no,2016-03-31,60437,2016-04-06 10:17:21


In [54]:
# distribution of sliced data
auto_valid['registration_year'].value_counts(normalize=True, dropna=False)[:20]

2000    0.068813
1999    0.063812
2005    0.062548
2006    0.056702
2001    0.056668
2003    0.055701
2004    0.055345
2002    0.053784
1998    0.050314
2007    0.049535
2008    0.045336
2009    0.043744
1997    0.041219
2010    0.034626
2011    0.033825
1996    0.030512
2016    0.027633
1995    0.027070
2012    0.026397
2013    0.017257
Name: registration_year, dtype: float64

Lets explore variations across different car brands.

In [57]:
# the distribution of the car brands
auto_valid['brand'].value_counts(normalize=True, dropna=False)

volkswagen        0.212391
bmw               0.109695
opel              0.107069
mercedes_benz     0.095955
audi              0.089352
ford              0.068849
renault           0.047559
peugeot           0.029859
fiat              0.025758
seat              0.018631
skoda             0.015407
mazda             0.015348
smart             0.014104
citroen           0.013874
nissan            0.013569
toyota            0.012747
sonstige_autos    0.010648
hyundai           0.009830
mini              0.009210
volvo             0.009129
mitsubishi        0.008266
honda             0.007587
kia               0.006878
alfa_romeo        0.006348
suzuki            0.006318
porsche           0.006149
chevrolet         0.005014
chrysler          0.003941
dacia             0.002450
daihatsu          0.002186
jeep              0.002186
land_rover        0.002136
subaru            0.002125
jaguar            0.001721
trabant           0.001620
saab              0.001452
daewoo            0.001438
r

In [67]:
# calculate the price of common brands
common = list(auto_valid['brand'].value_counts().index)

def mean_calc(brand_list, column):
    brand_mean = {}
    for car in brand_list:
        brand_mean[car] = auto_valid.loc[auto_valid['brand'] == car, column].mean()
    return brand_mean
mean_price = mean_calc(common, 'price')
# mean_price

sorted_dict = {}
sorted_keys = sorted(mean_price, key=mean_price.get) 

for w in sorted_keys:
    sorted_dict[w] = mean_price[w]

sorted_dict

{'daewoo': 1011.812865497076,
 'rover': 1524.3952483801295,
 'daihatsu': 1704.548717948718,
 'renault': 2366.3506011315417,
 'lada': 3030.3899082568805,
 'peugeot': 3206.2663099596357,
 'lancia': 3218.629067245119,
 'opel': 3248.27832460733,
 'mitsubishi': 3298.8646998982704,
 'smart': 3586.280802861685,
 'honda': 3863.605097894348,
 'saab': 3879.23166023166,
 'chrysler': 3925.278093883357,
 'fiat': 4146.933841131665,
 'subaru': 4247.2744063324535,
 'suzuki': 4401.918811002662,
 'seat': 4420.369038664059,
 'nissan': 4585.39599256352,
 'volvo': 5100.101627264354,
 'toyota': 5263.365875109938,
 'hyundai': 5487.679498146564,
 'kia': 5719.187041564792,
 'mazda': 5796.058619430241,
 'dacia': 5861.871853546911,
 'skoda': 6460.418046207022,
 'chevrolet': 7485.864169927334,
 'ford': 8702.476062530532,
 'citroen': 9167.919797979797,
 'mini': 9979.38496652465,
 'jeep': 12207.964102564103,
 'volkswagen': 13643.476727767,
 'jaguar': 14232.819218241042,
 'bmw': 14798.226103176023,
 'audi': 16218.25

Sonstige Autos, Porsche, Alfa Romeo, Trabant, Mercedes Benze, Land Rover, Audi, BMW, Jaguar & Volkswagen have the most expesive cars. This suggests that e-commerce sales mirror real-life prestige enjoyed by those brands. However, in the top 10 most common brands there's a lost of variation in mean price. The mean price of an opel is over 99% lower than that of an Sonstige Autos, (most expensive). However, even compared to Rover which is the the 2nd least expensive common car, the price of an opel is 34% lower. In order to investigate the variance in price, the average mileage of each of these brands will be investigated in order to see if this is the main factor affecting the price variability.

Let's derive the average mileage for those cars and check if there's any visible link with mean price.


In [69]:
# create a dataframe containing the average price and average mileage
mean_mileage = mean_calc(common, 'kilometer')
mean_price_series = pd.Series(mean_price)
mean_price_df = pd.DataFrame(mean_price_series,columns=['mean_price'])
mean_mileage_series = pd.Series(mean_mileage)
mean_mileage_df = pd.DataFrame(mean_mileage_series, columns=['mean_mileage'])
mean_price_df['mean_milage'] = mean_mileage_df['mean_mileage']

In [73]:
mean_price_df.sort_values('mean_price',ascending=False)[:10]

,mean_price,mean_milage
sonstige_autos,642492.477231,86637.272967
porsche,51301.345943,98318.140383
alfa_romeo,36896.854305,128688.741722
trabant,20828.153979,56046.712803
mercedes_benz,17614.749292,130580.692274
land_rover,16823.683727,119258.530184
audi,16218.259230,129491.201104
bmw,14798.226103,132657.076424
jaguar,14232.819218,121872.964169
volkswagen,13643.476728,128337.160352


In [ ]:
There seems to be no clear pattern between mean price and mean mileage this indicates that the most expensive cars sell for more money due to some other factors, such as perceived quality or durability.